## 0. Import Libraries

In [ ]:
# Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from scipy import stats
from scipy.stats import shapiro, normaltest, kstest, f_oneway
import warnings

warnings.filterwarnings('ignore')

# Academic publication style settings
plt.style.use('seaborn-v0_8-paper')
sns.set_context("paper", font_scale=1.5)
sns.set_palette("deep")

# Configure matplotlib for publication quality
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman', 'DejaVu Serif']
plt.rcParams['font.size'] = 13
plt.rcParams['axes.labelsize'] = 15
plt.rcParams['axes.titlesize'] = 17
plt.rcParams['axes.titleweight'] = 'bold'
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['xtick.labelsize'] = 13
plt.rcParams['ytick.labelsize'] = 13
plt.rcParams['legend.fontsize'] = 12
plt.rcParams['legend.frameon'] = True
plt.rcParams['legend.edgecolor'] = 'black'
plt.rcParams['axes.linewidth'] = 1.2
plt.rcParams['grid.alpha'] = 0.3
plt.rcParams['grid.linestyle'] = '--'

print("Libraries loaded successfully!")
print("Academic publication style configured")

## 1. Load Data

In [ ]:
# Load cleaned dataset with artist features
data_path = Path('../data/songs.csv')
print(f"Loading data from: {data_path}")

df = pd.read_csv(data_path)

print(f"\nDataset loaded successfully!")
print(f"Shape: {df.shape[0]:,} rows x {df.shape[1]} columns")

## 2. Statistical Tests for Normality

Test whether target variables follow normal distributions

In [ ]:
# Define targets
target_vars = ['valence', 'energy', 'danceability', 'popularity']

# Normality tests
print("="*80)
print("NORMALITY TESTS FOR TARGET VARIABLES")
print("="*80)

for target in target_vars:
    data = df[target].dropna()
    
    # Shapiro-Wilk test (best for n < 5000)
    sample = data.sample(n=min(5000, len(data)), random_state=42)
    shapiro_stat, shapiro_p = shapiro(sample)
    
    # D'Agostino-Pearson test
    dagostino_stat, dagostino_p = normaltest(data)
    
    # Kolmogorov-Smirnov test
    ks_stat, ks_p = kstest(data, 'norm', args=(data.mean(), data.std()))
    
    print(f"\n{target.upper()}:")
    print(f"  Shapiro-Wilk:     statistic={shapiro_stat:.4f}, p-value={shapiro_p:.4e}")
    print(f"  D'Agostino:       statistic={dagostino_stat:.4f}, p-value={dagostino_p:.4e}")
    print(f"  Kolmogorov-Smirnov: statistic={ks_stat:.4f}, p-value={ks_p:.4e}")
    
    if shapiro_p < 0.05:
        print(f"  NOT normally distributed (reject H0 at α=0.05)")
    else:
        print(f"  Normally distributed (fail to reject H0 at α=0.05)")

print("\n" + "="*80)

## 3. ANOVA: Genre Effects on Targets

Test whether genre significantly affects each target variable

In [ ]:
print("="*80)
print("ANOVA: GENRE EFFECTS ON TARGET VARIABLES")
print("="*80)

for target in target_vars:
    # Group by genre
    genre_groups = [df[df['genre'] == genre][target].dropna() for genre in df['genre'].unique()]
    
    # One-way ANOVA
    f_stat, p_value = f_oneway(*genre_groups)
    
    print(f"\n{target.upper()}:")
    print(f"  F-statistic: {f_stat:.4f}")
    print(f"  p-value: {p_value:.4e}")
    
    if p_value < 0.05:
        print(f"  Genre has SIGNIFICANT effect on {target} (reject H0 at α=0.05)")
    else:
        print(f"  Genre has NO significant effect on {target}")

print("\n" + "="*80)

## 4. Interaction Effects: Genre × Year

Analyze how target variables change over time for different genres

In [ ]:
# Focus on modern era (1970+) with sufficient data
df_modern = df[df['year'] >= 1970].copy()

# Get top 5 genres by count
top_genres = df_modern['genre'].value_counts().head(5).index

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Genre × Year Interaction Effects (1970-2025)', fontsize=18, fontweight='bold', y=0.995)

# Color palette for genres
colors = plt.cm.Set2(np.linspace(0, 1, len(top_genres)))

for idx, target in enumerate(target_vars):
    ax = axes[idx // 2, idx % 2]
    
    for i, genre in enumerate(top_genres):
        genre_data = df_modern[df_modern['genre'] == genre]
        year_means = genre_data.groupby('year')[target].mean()
        
        ax.plot(year_means.index, year_means.values, marker='o', markersize=4,
                label=genre, linewidth=2.5, alpha=0.8, color=colors[i])
    
    ax.set_xlabel('Year', fontsize=13, fontweight='bold')
    # ax.set_ylabel(target.capitalize(), fontsize=13, fontweight='bold')
    ax.set_title(f'{target.capitalize()}', fontsize=15, fontweight='bold', pad=12)
    ax.legend(loc='best', frameon=True, shadow=True, ncol=1)
    ax.grid(True, alpha=0.3, linestyle='--')
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

plt.tight_layout()
plt.savefig('../results/figures/eda/genre_year_interaction.png', dpi=300, bbox_inches='tight')
plt.show()

## 5. Pair Plot: Top Correlated Features

Visualize relationships between highly correlated audio features

In [ ]:
# Select features with highest correlations to valence
audio_features = [
    'acousticness', 'instrumentalness', 'liveness', 'speechiness',
    'loudness', 'tempo', 'duration_ms', 'danceability', 'energy'
]

# Get top 4 features correlated with valence
valence_corr = df[audio_features + ['valence']].corr()['valence'].drop('valence').abs().sort_values(ascending=False)
top_features = valence_corr.head(4).index.tolist()

# Add valence for comparison
pair_cols = top_features + ['valence']

# Sample for performance (pair plot is slow)
df_sample = df[pair_cols].sample(n=min(5000, len(df)), random_state=42)

# Create pair plot with academic styling
g = sns.pairplot(df_sample, diag_kind='kde', 
                 plot_kws={'alpha': 0.5, 's': 25, 'edgecolor': 'none'},
                 diag_kws={'linewidth': 2.5, 'alpha': 0.7})

g.fig.suptitle('Pairwise Feature Relationships with Valence', 
               y=1.01, fontsize=16, fontweight='bold')

# Style each subplot
for ax in g.axes.flatten():
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.grid(True, alpha=0.25, linestyle='--')
    ax.tick_params(labelsize=9)

plt.savefig('../results/figures/eda/pairplot_valence.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n" + "="*80)
print("TABLE 3: Top Features Correlated with Valence")
print("="*80)
for feature, corr in valence_corr.head(4).items():
    print(f"  {feature:20s}: r = {corr:.3f}")
print("="*80)

## 6. Skewness & Kurtosis Analysis

Verify which features need transformation

In [ ]:
from scipy.stats import skew, kurtosis

# Calculate skewness and kurtosis for all numeric features
numeric_cols = df.select_dtypes(include=[np.number]).columns

skewness_data = []
for col in numeric_cols:
    skew_val = skew(df[col].dropna())
    kurt_val = kurtosis(df[col].dropna())
    skewness_data.append({
        'Feature': col,
        'Skewness': skew_val,
        'Kurtosis': kurt_val,
        'Transformation': 'Power/Log' if abs(skew_val) > 1.0 else 'StandardScaler'
    })

skewness_df = pd.DataFrame(skewness_data).sort_values('Skewness', key=abs, ascending=False)

print("="*80)
print("SKEWNESS & KURTOSIS ANALYSIS")
print("="*80)
print("\nInterpretation:")
print("  Skewness < -1 or > 1: Highly skewed (needs transformation)")
print("  Skewness between -0.5 and 0.5: Roughly symmetric")
print("  Kurtosis > 3: Heavy tails (many outliers)")
print("\n")
print(skewness_df.to_string(index=False))
print("\n" + "="*80)

## 7. Artist Features: Distribution Analysis

Analyze the new artist features for Experiment 2

In [ ]:
# Artist features distribution
fig, axes = plt.subplots(1, 2, figsize=(16, 6))
fig.suptitle('Artist Features Distribution', fontsize=20, fontweight='bold', y=0.995)

# Total Artist Followers (clean, trimmed, linear scale)
ax = axes[0]
followers = df['total_artist_followers'].dropna()

# Optional: exclude zero followers if they distort the view (common in Spotify data)
followers_nonzero = followers[followers > 0]

# Use many thin bins for detail
ax.hist(followers_nonzero, bins=100, edgecolor='black', alpha=0.75,
        color='#3498db', linewidth=1.0)

mean_val = followers_nonzero.mean()
ax.axvline(mean_val, color='darkred', linestyle='--', linewidth=2.5,
           label=f'Mean = {mean_val:,.0f}', alpha=0.9)

ax.set_xlim(right=followers_nonzero.quantile(0.99))
ax.set_xlim(left=0)

# Optional: Add a note that the plot is trimmed
ax.text(0.98, 0.95, 'Trimmed at 99th percentile', transform=ax.transAxes,
        fontsize=11, verticalalignment='top', horizontalalignment='right',
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.7))

# ax.set_xlabel('Total Artist Followers', fontsize=17, fontweight='bold')
# ax.set_ylabel('Frequency', fontsize=17, fontweight='bold')
ax.set_title('Total Artist Followers', fontsize=18, fontweight='bold', pad=12)
ax.legend(loc='upper right', frameon=True, shadow=True, fontsize=13)
ax.grid(True, alpha=0.3, linestyle='--')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.tick_params(axis='both', labelsize=14)
ax.ticklabel_format(style='plain', axis='x')  # Show full numbers, no scientific notation

# Average Artist Popularity (unchanged - already looks great)
ax = axes[1]
popularity = df['avg_artist_popularity'].dropna()
ax.hist(popularity, bins=50, edgecolor='black', alpha=0.75,
        color='#e74c3c', linewidth=1.2)
mean_val = popularity.mean()
ax.axvline(mean_val, color='darkred', linestyle='--', linewidth=2.5,
           label=f'Mean = {mean_val:.2f}', alpha=0.9)
# ax.set_xlabel('Average Artist Popularity', fontsize=17, fontweight='bold')
# ax.set_ylabel('Frequency', fontsize=17, fontweight='bold')
ax.set_title('Average Artist Popularity', fontsize=18, fontweight='bold', pad=12)
ax.legend(loc='upper right', frameon=True, shadow=True, fontsize=13)
ax.grid(True, alpha=0.3, linestyle='--')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.tick_params(axis='both', labelsize=14)

plt.tight_layout()
plt.savefig('../results/figures/eda/artist_features_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n" + "="*80)
print("Artist Features Statistics:")
print("="*80)
print(f"Total Artist Followers:")
print(f"  Mean: {df['total_artist_followers'].mean():,.0f}")
print(f"  Median: {df['total_artist_followers'].median():,.0f}")
print(f"  99th Percentile: {df['total_artist_followers'].quantile(0.99):,.0f}")
print(f"  Max: {df['total_artist_followers'].max():,.0f}")
print(f"\nAverage Artist Popularity:")
print(f"  Mean: {df['avg_artist_popularity'].mean():.2f}")
print(f"  Median: {df['avg_artist_popularity'].median():.2f}")
print(f"  Max: {df['avg_artist_popularity'].max():.2f}")
print("="*80)

## 8. Artist Features: Impact on Targets

Analyze how artist fame affects song characteristics

In [ ]:
# Create artist fame categories based on followers
df['artist_fame'] = pd.qcut(df['total_artist_followers'], 
                             q=5, 
                             labels=['Very Low', 'Low', 'Medium', 'High', 'Very High'],
                             duplicates='drop')

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Target Variables by Artist Fame Level', 
             fontsize=20, fontweight='bold', y=0.995)

for idx, target in enumerate(target_vars):
    ax = axes[idx // 2, idx % 2]
    
    # Box plot by artist fame
    data_by_fame = [df[df['artist_fame'] == level][target].dropna() 
                    for level in ['Very Low', 'Low', 'Medium', 'High', 'Very High']]
    
    bp = ax.boxplot(data_by_fame, labels=['Very Low', 'Low', 'Medium', 'High', 'Very High'],
                    patch_artist=True,
                    boxprops=dict(facecolor='lightblue', edgecolor='black', linewidth=1.3),
                    medianprops=dict(color='darkred', linewidth=2.5),
                    whiskerprops=dict(color='black', linewidth=1.3),
                    capprops=dict(color='black', linewidth=1.3),
                    flierprops=dict(marker='o', markerfacecolor='red', markersize=3, 
                                   alpha=0.4, markeredgecolor='darkred'))
    
    # ax.set_xlabel('Artist Fame Level', fontsize=17, fontweight='bold')
    # ax.set_ylabel(target.capitalize(), fontsize=17, fontweight='bold')
    ax.set_title(f'{target.capitalize()}', fontsize=18, fontweight='bold', pad=12)
    ax.tick_params(axis='x', rotation=45, labelsize=13)
    ax.tick_params(axis='y', labelsize=14)
    ax.grid(True, alpha=0.3, axis='y', linestyle='--')
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

plt.tight_layout()
plt.savefig('../results/figures/eda/targets_by_artist_fame.png', dpi=300, bbox_inches='tight')
plt.show()

# Calculate correlation and ANOVA
print("\n" + "="*80)
print("Artist Fame Impact Analysis")
print("="*80)

for target in target_vars:
    # Correlation with raw followers
    corr = df[['total_artist_followers', target]].dropna().corr().iloc[0, 1]
    
    # ANOVA across fame levels
    fame_groups = [df[df['artist_fame'] == level][target].dropna() 
                   for level in ['Very Low', 'Low', 'Medium', 'High', 'Very High']]
    f_stat, p_value = f_oneway(*fame_groups)
    
    print(f"\n{target.upper()}:")
    print(f"  Correlation with followers: r = {corr:.4f}")
    print(f"  ANOVA F-statistic: {f_stat:.2f}, p-value: {p_value:.4e}")
    if p_value < 0.05:
        print(f"  Significant effect (p < 0.05)")
    else:
        print(f"  No significant effect")

print("\n" + "="*80)

## 9. Artist Metrics by Genre

Compare artist fame across different genres

In [ ]:
# Artist popularity distribution by genre
top_5_genres = df['genre'].value_counts().head(5).index

fig, axes = plt.subplots(1, 2, figsize=(16, 6))
# fig.suptitle('Artist Metrics by Genre', fontsize=20, fontweight='bold', y=0.995)

# Artist followers by genre (log scale for visualization)
ax = axes[0]
data_by_genre = [df[df['genre'] == g]['total_artist_followers'].dropna() 
                 for g in top_5_genres]
bp = ax.boxplot(data_by_genre, labels=top_5_genres, patch_artist=True,
                boxprops=dict(facecolor='#3498db', edgecolor='black', linewidth=1.3),
                medianprops=dict(color='darkred', linewidth=2.5),
                whiskerprops=dict(color='black', linewidth=1.3),
                capprops=dict(color='black', linewidth=1.3),
                flierprops=dict(marker='o', markerfacecolor='red', markersize=3, 
                               alpha=0.4, markeredgecolor='darkred'))
# ax.set_xlabel('Genre', fontsize=17, fontweight='bold')
# ax.set_ylabel('Total Artist Followers', fontsize=17, fontweight='bold')
ax.set_title('Artist Followers by Genre', fontsize=18, fontweight='bold', pad=12)
ax.set_yscale('log')  # Log scale for y-axis
ax.tick_params(axis='x', rotation=45, labelsize=13)
ax.tick_params(axis='y', which='minor', left=False, right=False, labelsize=14)
ax.grid(True, alpha=0.3, axis='y', linestyle='--')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Artist popularity by genre
ax = axes[1]
data_by_genre = [df[df['genre'] == g]['avg_artist_popularity'].dropna() 
                 for g in top_5_genres]
bp = ax.boxplot(data_by_genre, labels=top_5_genres, patch_artist=True,
                boxprops=dict(facecolor='#e74c3c', edgecolor='black', linewidth=1.3),
                medianprops=dict(color='darkred', linewidth=2.5),
                whiskerprops=dict(color='black', linewidth=1.3),
                capprops=dict(color='black', linewidth=1.3),
                flierprops=dict(marker='o', markerfacecolor='red', markersize=3, 
                               alpha=0.4, markeredgecolor='darkred'))
# ax.set_xlabel('Genre', fontsize=17, fontweight='bold')
# ax.set_ylabel('Average Artist Popularity', fontsize=17, fontweight='bold')
ax.set_title('Artist Popularity by Genre', fontsize=18, fontweight='bold', pad=12)
ax.tick_params(axis='x', rotation=45, labelsize=13)
ax.tick_params(axis='y', labelsize=14)
ax.grid(True, alpha=0.3, axis='y', linestyle='--')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.tight_layout()
plt.savefig('../results/figures/eda/artist_metrics_by_genre.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n" + "="*80)
print("Artist Metrics by Genre:")
print("="*80)
for genre in top_5_genres:
    genre_data = df[df['genre'] == genre]
    mean_followers = genre_data['total_artist_followers'].mean()
    mean_popularity = genre_data['avg_artist_popularity'].mean()
    print(f"{genre:15s}: Followers={mean_followers:12,.0f}, Popularity={mean_popularity:5.2f}")
print("="*80)